In [1]:
cd ..

C:\Users\agus\Dropbox\Doctorado\NeuroData


In [2]:
import PyLeech.Utils.NLDUtils as NLD
import PyLeech.Utils.AbfExtension as abfe

import PyLeech.Utils.CrawlingDatabaseUtils as CDU
import PyLeech.Utils.burstStorerLoader as bStorerLoader
import PyLeech.Utils.burstUtils as burstUtils

import numpy as np
import scipy.signal as spsig
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
%matplotlib qt5

In [1]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}
matplotlib.rcParams['figure.figsize'] = (20, 15)
matplotlib.rc('font', **font)

NameError: name 'matplotlib' is not defined

In [4]:
cdd = CDU.loadDataDict()
cdb = CDU.loadCrawlingDatabase()
file_list = []
for files in list(cdd.keys()):
    burst_obj = bStorerLoader.BurstStorerLoader(files, 'RegistrosDP_PP', mode='load')
    try:
        arr_dict, time_vector1, fs = abfe.getArraysFromAbfFiles(files, ['Vm1'])
        file_list.append(files)
    except:
        pass



channel Vm1 doesn't exists, check ch_info: 
{'id': 5, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 0}
{'id': 6, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 1}
Deleting block
channel Vm1 doesn't exists, check ch_info: 
{'id': 5, 'sampling_rate': 15151.515151515154, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 0}
{'id': 6, 'sampling_rate': 15151.515151515154, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 1}
Deleting block
I need to set attribute 'isDe3' before continuing
I need to set attribute 'isDe3' before continuing


In [5]:
binning_dt = 0.1
kernel_sigma = 5

#### NS

In [6]:
for fn in [file_list[0]]:
    arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, ['Vm1'])
    NS = arr_dict['Vm1']
    del arr_dict


    NS[(NS > -20) | (NS < -60)] = np.median(NS)
    good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

    crawling_intervals = cdd[fn]['crawling_intervals']
    print(crawling_intervals)

    burst_obj = bStorerLoader.BurstStorerLoader(fn, 'RegistrosDP_PP', mode='load')
    
    binning_dt = 0.1
#     kernel_sigma = 5
    kernel_sigma = 5
    new_sfd = burstUtils.removeOutliers(burst_obj.spike_freq_dict, 5)
    binned_sfd = burstUtils.digitizeSpikeFreqs(new_sfd, binning_dt, time_vector[-1], count=False)
    cut_binned_freq_array = burstUtils.binned_spike_freq_dict_ToArray(binned_sfd, crawling_intervals, good_neurons)
    
    
    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=20, dt_step=binning_dt)
    smoothed_sfd = {}
    for key, items in binned_sfd.items():
        smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])
    
    step = int(binning_dt * fs)
    neuron_idxs = []
    for interval in crawling_intervals:
        neuron_idxs.append(np.where((time_vector > interval[0]) & (time_vector < interval[1]))[0][::step])
    neuron_idxs = np.hstack(neuron_idxs)

    neuron_cut_time = time_vector[neuron_idxs]

    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=30, dt_step=1 / fs)
    conv_NS = spsig.fftconvolve(NS, kernel, mode='same')
    cut_NS = conv_NS[neuron_idxs]

#     fig, axes = burstUtils.plotFreq(smoothed_sfd, color_dict=burst_obj.color_dict,
#                                     template_dict=burst_obj.template_dict, scatter_plot=False,
#                                     optional_trace=[time_vector[::int(step / 4)], conv_NS[::int(step / 4)]],
#                                     outlier_thres=None, ms=4)
#     fig.suptitle(fn)

#     NS_embedding = NLD.getDerivativeEmbedding(cut_NS, 0.1, 3)
    
    NS_embedding = NLD.getTraceEmbedding(cut_NS, int(5 / .1), 3)
    jumps = np.where(np.diff(neuron_idxs) != np.diff(neuron_idxs)[0])[0]
    if len(jumps) > 0:
        NS_embedding[jumps] = np.nan
        NS_embedding[jumps + 1] = np.nan

#     scaler = StandardScaler()
#     NS_embedding = scaler.fit_transform(NS_embedding)

    fig, ax = NLD.plot3Dline(NS_embedding, color_idx=None)
    fig.suptitle(fn)
#     NS_embedding = NLD.getTraceEmbedding(cut_NS[:], int(1.2 / .1), 3)
    rt1 = NLD.getCloseReturns(NS_embedding)

#     reord_rt, fig1, ax1, 
    rt, fig1, ax1, fig2, ax2 = NLD.plotCloseReturns(rt1, get_counts=True, reorder=True, masked=False, thr=.1, return_reordered=True)
    fig1.suptitle(fn)
    fig2.suptitle(fn)

[[350, 700]]
I need to set attribute 'isDe3' before continuing


TypeError: digitizeSpikeFreqs() got an unexpected keyword argument 'count'

In [ ]:
for fn in [file_list[0]]:
    arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, ['Vm1'])
    NS = arr_dict['Vm1']
    del arr_dict

    NS[(NS > -20) | (NS < -60)] = np.median(NS)
    good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

    crawling_intervals = cdd[fn]['crawling_intervals']
    print(crawling_intervals)

    burst_obj = bStorerLoader.BurstStorerLoader(fn, 'RegistrosDP_PP', mode='load')
    


    new_sfd = burstUtils.removeOutliers(burst_obj.spike_freq_dict, 5)
    binned_sfd = burstUtils.digitizeSpikeFreqs(new_sfd, binning_dt, time_vector[-1], count=False)
    cut_binned_freq_array = burstUtils.binned_spike_freq_dict_ToArray(binned_sfd, crawling_intervals, good_neurons)
    
    
    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=20, dt_step=binning_dt)
    smoothed_sfd = {}
    for key, items in binned_sfd.items():
        smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])
    
    step = int(binning_dt * fs)
    neuron_idxs = []
    for interval in crawling_intervals:
        neuron_idxs.append(np.where((time_vector > interval[0]) & (time_vector < interval[1]))[0][::step])
    neuron_idxs = np.hstack(neuron_idxs)

    neuron_cut_time = time_vector[neuron_idxs]

    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=30, dt_step=1 / fs)
    conv_NS = spsig.fftconvolve(NS, kernel, mode='same')
    cut_NS = conv_NS[neuron_idxs]

    
#     NS_embedding = NLD.getDerivativeEmbedding(cut_NS, 0.1, 3)
    NS_embedding = NLD.getTraceEmbedding(cut_NS, int(5 / .1), 3)
    jumps = np.where(np.diff(neuron_idxs) != np.diff(neuron_idxs)[0])[0]
    if len(jumps) > 0:
        NS_embedding[jumps] = np.nan
        NS_embedding[jumps + 1] = np.nan
    
    scaler = StandardScaler()
    NS_embedding = scaler.fit_transform(NS_embedding)
    
#     NLD.plot3Dline(NS_embedding)
    rt1 = NLD.getCloseReturns(NS_embedding)


    for pair in [[800, 1700], [1900, 2450]]:
        fig, ax = NLD.plot3Dline(NS_embedding[pair[0]:pair[1]], color_idx=None)
        ax.set_title(fn + '\n' + str(pair[0]) + '-' + str(pair[1]))
    
    
    
    fig, ax = NLD.plot3Dline(NS_embedding[150:300], color_idx=None)
    NLD.plot3Dline(NS_embedding[150+1400:300+1400], color_idx=None)
        

In [ ]:
loops_dict = {
    file_list[0]:
        [
            [180, 300+1050],
            [520,750+550],
            [850, 1050+350],
            [1260, 1400+350]
        ],
    file_list[1]:
        [
            [70,150+290],
            [350, 450+140],
            
        ],
    file_list[2]:
    [
        [750, 900+335],
        [750, 950+1250],
        [750, 750 + 725],
        [2220, 3000+100],
        [1550, 1700+800],
        [3280, 3400+280]
    ],
    file_list[3]:
        [
            [300, 800],
            [1300, 1350+570],
            [2200, 2500+225],
            [2410, 2700+370],
            [2450, 2500+550],
            [170, 250+500],
            
    
        ]      
}
loops_dict

In [ ]:
for fn in file_list:
    arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, ['Vm1'])
    NS = arr_dict['Vm1']
    del arr_dict


    NS[(NS > -20) | (NS < -60)] = np.median(NS)
    good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

    crawling_intervals = cdd[fn]['crawling_intervals']
    print(crawling_intervals)

    burst_obj = bStorerLoader.BurstStorerLoader(fn, 'RegistrosDP_PP', mode='load')
    


    new_sfd = burstUtils.removeOutliers(burst_obj.spike_freq_dict, 5)
    binned_sfd = burstUtils.digitizeSpikeFreqs(new_sfd, binning_dt, time_vector[-1], count=False)
    cut_binned_freq_array = burstUtils.binned_spike_freq_dict_ToArray(binned_sfd, crawling_intervals, good_neurons)
    
    
    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=20, dt_step=binning_dt)
    smoothed_sfd = {}
    for key, items in binned_sfd.items():
        smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])

    
    
    step = int(binning_dt * fs)
    neuron_idxs = []
    for interval in crawling_intervals:
        neuron_idxs.append(np.where((time_vector > interval[0]) & (time_vector < interval[1]))[0][::step])
    neuron_idxs = np.hstack(neuron_idxs)

    neuron_cut_time = time_vector[neuron_idxs]

    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=30, dt_step=1 / fs)
    conv_NS = spsig.fftconvolve(NS, kernel, mode='same')
    cut_NS = conv_NS[neuron_idxs]

    
    NS_embedding = NLD.getDerivativeEmbedding(cut_NS, 0.1, 3)
    jumps = np.where(np.diff(neuron_idxs) != np.diff(neuron_idxs)[0])[0]
    if len(jumps) > 0:
        NS_embedding[jumps] = np.nan
        NS_embedding[jumps + 1] = np.nan
    
    scaler = StandardScaler()
    NS_embedding = scaler.fit_transform(NS_embedding)
    
#     NLD.plot3Dline(NS_embedding)
    rt1 = NLD.getCloseReturns(NS_embedding)


    for pair in loops_dict[fn]:
        fig, ax = NLD.plot3Dline(NS_embedding[pair[0]:pair[1]])
        ax.set_title(fn + '\n' + str(pair[0]) + '-' + str(pair[1]))

#### Neuronas

In [ ]:
for fn in file_list:
    arr_dict, time_vector, fs = abfe.getArraysFromAbfFiles(fn, ['Vm1'])
    NS = arr_dict['Vm1']
    del arr_dict


    NS[(NS > -20) | (NS < -60)] = np.median(NS)
    good_neurons = [neuron for neuron, neuron_dict in cdd[fn]['neurons'].items() if neuron_dict['neuron_is_good']]

    crawling_intervals = cdd[fn]['crawling_intervals']
    print(crawling_intervals)

    burst_obj = bStorerLoader.BurstStorerLoader(fn, 'RegistrosDP_PP', mode='load')
    
    binning_dt = 0.1
    kernel_sigma = 1
    
    new_sfd = burstUtils.removeOutliers(burst_obj.spike_freq_dict, 5)
    binned_sfd = burstUtils.digitizeSpikeFreqs(new_sfd, binning_dt, time_vector[-1], count=False)
    cut_binned_freq_array = burstUtils.binned_spike_freq_dict_ToArray(binned_sfd, crawling_intervals, good_neurons)
    
    
    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=20, dt_step=binning_dt)
    smoothed_sfd = {}
    for key, items in binned_sfd.items():
        smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])
    
    step = int(binning_dt * fs)
    neuron_idxs = []
    for interval in crawling_intervals:
        neuron_idxs.append(np.where((time_vector > interval[0]) & (time_vector < interval[1]))[0][::step])
    neuron_idxs = np.hstack(neuron_idxs)

    neuron_cut_time = time_vector[neuron_idxs]

    kernel = NLD.generateGaussianKernel(sigma=kernel_sigma, time_range=30, dt_step=1 / fs)
    conv_NS = spsig.fftconvolve(NS, kernel, mode='same')
    cut_NS = conv_NS[neuron_idxs]

    fig, axes = burstUtils.plotFreq(smoothed_sfd, color_dict=burst_obj.color_dict,
                                    template_dict=burst_obj.template_dict, scatter_plot=False,
                                    optional_trace=[time_vector[::int(step / 4)], conv_NS[::int(step / 4)]],
                                    outlier_thres=None, ms=4)
    fig.suptitle(fn)
    for key, items in smoothed_sfd.items():
        if cdd[fn]['neurons'][key]['neuron_is_good']:
            embedding = NLD.getDerivativeEmbedding(items[1], 0.1, 3)
            jumps = np.where(np.diff(neuron_idxs) != np.diff(neuron_idxs)[0])[0]
            if len(jumps) > 0:
                NS_embedding[jumps] = np.nan
                NS_embedding[jumps + 1] = np.nan

            scaler = StandardScaler()
            embedding = scaler.fit_transform(embedding)

            fig, ax = NLD.plot3Dline(NS_embedding)
            fig.suptitle(fn + '\n' + str(key))
    #     NS_embedding = NLD.getTraceEmbedding(cut_NS[:], int(1.2 / .1), 3)
            rt1 = NLD.getCloseReturns(NS_embedding)

    #     reord_rt, fig1, ax1, 
            rt, fig1, ax1, fig2, ax2 = NLD.plotCloseReturns(rt1, get_counts=True, reorder=True, masked=False, thr=.1, return_reordered=True)
    #     fig1.suptitle(fn)
            fig2.suptitle(fn + '\n' + str(key))